# Agenda: Dates and times in Pandas!

1. Dates and times in programming
2. Parsing dates with Pandas
3. The `.dt` accessor
4. Loading multiple files into a single data frame
5. How can you parse odd date formats?
6. Grouping with dates
7. Time deltas
8. Time series
9. Resampling

# Dates and times in programming

The most natural (and reasonable) way to 